<h1 align="left"><font color='red'><b>TRAFFIC SIGN RECOGNITION</b></font></h1>

<h2 align="left"> <font color='lavander'>Import neccessary libraries</font></h2>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-04-22 00:38:34.809646: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-22 00:38:34.849315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-22 00:38:34.850252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 00:38:39.551040: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<h2 align="left"> <font color='lavander'>Generate train image data</font></h2>

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('train_data/_annotations.csv')

train_labels['class'] = train_labels['class'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='train_data',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

<h2 align="left"> <font color='lavander'>Pre trained model - ResNet50</font></h2>

In [ ]:
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

#freeze Layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

94765736/94765736 [==============================] - 23s 0us/step


2023-04-20 22:08:07.649609: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/20
3/3 [==============================] - 6s 1s/step - loss: 0.4196 - accuracy: 0.8571
Epoch 2/20
3/3 [==============================] - 4s 1s/step - loss: 0.4198 - accuracy: 0.8571
Epoch 3/20
3/3 [==============================] - 4s 992ms/step - loss: 0.4113 - accuracy: 0.8571
Epoch 4/20
3/3 [==============================] - 4s 953ms/step - loss: 0.4145 - accuracy: 0.8571
Epoch 5/20
3/3 [==============================] - 4s 1s/step - loss: 0.4107 - accuracy: 0.8571
Epoch 6/20
3/3 [==============================] - 3s 950ms/step - loss: 0.4122 - accuracy: 0.8571
Epoch 7/20
3/3 [==============================] - 4s 1s/step - loss: 0.4108 - accuracy: 0.8571
Epoch 8/20
3/3 [==============================] - 4s 1s/step - loss: 0.4110 - accuracy: 0.8571
Epoch 9/20
3/3 [==============================] - 3s 1s/step - loss: 0.4107 - accuracy: 0.8571
Epoch 10/20
3/3 [==============================] - 4s 1s/step - loss: 0.4105 - accuracy: 0.8571
Epoch 11/20
3/3 [=======================

In [ ]:
t_test_df = pd.read_csv("train_data/_annotations.csv")

<h2 align="left"> <font color='lavander'>Generate test image data</font></h2>

In [ ]:
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="train_data",
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['filename'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['filename'].isin(missing_filenames)]

Found 118 validated image filenames.


/home/u190385/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 464 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/home/u190385/tmp/ipykernel_3064396/554181950.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)
2023-04-20 22:09:29.428152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [ ]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'filename': t_test_generator.filenames[:len(preds)], 'class': li})

test_inst_df=pd.merge(df, t_test_df, on='filename', how='inner')

<h2 align="left"> <font color='lavander'>Save the model</font></h2>

In [17]:
model.save('models/resnet.h5')